In [2]:
import pandas as pd
import kagglehub
import os
import kagglehub
import os
import shutil
import pandas as pd

/Users/giliamo/.pyenv/versions/3.9.7/envs/automated_loan_review_project/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
# Download latest version
path = kagglehub.dataset_download("yasserh/loan-default-dataset")
print("Path to dataset files:", path)

Path to dataset files: /Users/giliamo/.cache/kagglehub/datasets/yasserh/loan-default-dataset/versions/1


In [18]:
import kagglehub
import os
import shutil

# Download latest version
path = kagglehub.dataset_download("yasserh/loan-default-dataset")
print("Path to dataset files:", path)

# Set the target directory for storing the specific CSV file
target_dir = "/Users/giliamo/code/YannAll/raw_data"

# Create the target directory if it does not exist
os.makedirs(target_dir, exist_ok=True)

# Specify the name of the CSV file you want to move
csv_file_name = "Loan_Default.csv"

# Construct the source path to the CSV file
source_csv_path = os.path.join(path, csv_file_name)

# Construct the target path for the CSV file
target_csv_path = os.path.join(target_dir, csv_file_name)

# Check if the CSV file exists in the downloaded path
if os.path.exists(source_csv_path):
    # Move the CSV file to the target directory
    shutil.move(source_csv_path, target_csv_path)
    print(f"CSV file has been moved to: {target_csv_path}")
else:
    print(f"CSV file '{csv_file_name}' not found in the downloaded dataset directory.")

# Load the CSV file using Pandas if it exists
import pandas as pd

if os.path.exists(target_csv_path):
    data = pd.read_csv(target_csv_path)
    print("CSV file loaded successfully.")
else:
    print(f"Could not load CSV file. File '{csv_file_name}' does not exist at: {target_csv_path}")


Path to dataset files: /Users/giliamo/.cache/kagglehub/datasets/yasserh/loan-default-dataset/versions/1
CSV file 'Loan_Default.csv' not found in the downloaded dataset directory.
Could not load CSV file. File 'Loan_Default.csv' does not exist at: /Users/giliamo/code/YannAll/raw_data/Loan_Default.csv


In [ ]:
data.head()